In [1]:
from ipycanvas import RoughCanvas
canvas = RoughCanvas()

canvas.stroke_rect(100, 100, 100, 100)
canvas.fill_rect(50, 50, 100, 100)

canvas.stroke_circle(300, 300, 100)
canvas.fill_circle(350, 350, 100)

canvas.stroke_line(200, 200, 300, 300)

canvas


RoughCanvas()

In [5]:
from math import pi

#c = RoughCanvas(width=600, height=600)
c = RoughCanvas()

c.fill_style = 'green'
c.fill_arc(300, 300, 200, 0, 0.9 * pi)

c.fill_style = 'red'
c.fill_arc(300, 300, 200, 0.9 * pi, pi + 0.2 * pi)

c.fill_style = 'blue'
c.fill_arc(300, 300, 200, pi + 0.2 * pi, 2 * pi)

c.stroke_style = 'black'
c.stroke_arc(300, 300, 200, 0, 2 * pi)

c

RoughCanvas()